
# Detecting Malicious Linux Reverse Shell Commands using Random Forest

This notebook trains a **Random Forest** classifier to identify potentially malicious
Linux command lines (reverse shells) and applies it to EDR logs.

Labels:
- **0**: Legitimate command
- **1**: Malicious (Reverse Shell)



## 1. Imports and Setup


In [1]:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix



## 2. Load Training Dataset

`dataset_linuxcommands.csv` contains:
- `command`
- `label` (0 = legitimate, 1 = malicious)


In [2]:

train_df = pd.read_csv("dataset_linuxcommands.csv")

print("Dataset shape:", train_df.shape)
train_df.head()


Dataset shape: (300, 2)


,command,label
0,cd /var/log,0
1,pwd,0
2,ls -la,0
3,cp file.txt /backup,0
4,mv oldname.txt newname.txt,0



## 3. Train/Test Split


In [3]:

X = train_df["command"]
y = train_df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)



## 4. Build Random Forest Pipeline

Steps:
- TF-IDF Vectorization
- Random Forest Classifier


In [4]:

pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        ngram_range=(1, 3),
        max_features=5000,
        lowercase=True
    )),
    ("rf", RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        n_jobs=-1,
        class_weight="balanced"
    ))
])



## 5. Train the Model


In [5]:

pipeline.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('rf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None



## 6. Evaluate Model Performance


In [6]:

y_pred = pipeline.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
confusion_matrix(y_test, y_pred)


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        30
           1       1.00      1.00      1.00        30

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60

Confusion Matrix:


array([[30,  0],
       [ 0, 30]])


## 7. Load EDR Command-Line Logs

`edr.csv` contains:
- `event.time`
- `src.process.cmdline`


In [7]:

edr_df = pd.read_csv("edr.csv")

print("EDR sample size:", edr_df.shape)
edr_df.head()


EDR sample size: (51834, 2)


,src.process.cmdline,nb
0,/bin/bash,7782
1,/bin/bash /etc/kvedr_server/inst_dir/storage/b...,101356
2,/bin/bash /opt/kafka_2.13-3.5.1/bin/kafka-brok...,7503
3,/bin/bash /opt/kafka_2.13-3.5.1/bin/kafka-run-...,30223
4,/bin/bash /usr/bin/version,25



## 8. Predict Malicious Commands in EDR Logs


In [8]:

edr_df["prediction"] = pipeline.predict(edr_df["src.process.cmdline"])
edr_df["malicious_probability"] = pipeline.predict_proba(
    edr_df["src.process.cmdline"]
)[:, 1]

edr_df.head()


,src.process.cmdline,nb,prediction,malicious_probability
0,/bin/bash,7782,0,0.210
1,/bin/bash /etc/kvedr_server/inst_dir/storage/b...,101356,0,0.215
2,/bin/bash /opt/kafka_2.13-3.5.1/bin/kafka-brok...,7503,0,0.215
3,/bin/bash /opt/kafka_2.13-3.5.1/bin/kafka-run-...,30223,0,0.215
4,/bin/bash /usr/bin/version,25,0,0.210



## 9. Extract Potential Reverse Shells


In [9]:

suspicious = edr_df[edr_df["prediction"] == 1]

print("Potential malicious commands detected:", suspicious.shape[0])
suspicious


Potential malicious commands detected: 0


,src.process.cmdline,nb,prediction,malicious_probability



## 10. Save Detection Results


In [10]:

suspicious.to_csv("edr_suspected_reverse_shells_rf.csv", index=False)
print("Results saved to edr_suspected_reverse_shells_rf.csv")


Results saved to edr_suspected_reverse_shells_rf.csv



## Conclusion

This notebook demonstrated how to:
- Train a Random Forest model on Linux command-line data
- Detect potential reverse shell payloads in EDR logs
- Export suspicious findings for further investigation
